Data familiriazation, cleaning, and preparing tools, tips, and tricks.


In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv(r'C:\GitHub\Contract_Profiles\data\interim\acc_ri_interim_data.csv')
# obsp_forecast_df = pd.read_csv(r'C:\PyProjects\OSBP Insights\references\forecast_listing\osbp_dashboard_forecast.csv')
# amc_forecast_df = pd.read_csv(r'C:\PyProjects\OSBP Insights\references\forecast_listing\amc_forecast_listing.csv')

In [3]:
# Display the number of rows and columns in the cleaned dataframe
df.shape

(29329, 55)

In [4]:
# Filter acc_ri_awards_df to not inlcude any rows with a value of 'MODFICATION", "SATOC", AND "MATOC" in the 'Contract Action Type' column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29329 entries, 0 to 29328
Data columns (total 55 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Contract No                              29329 non-null  object 
 1   Order No                                 29329 non-null  object 
 2   Modification No                          29329 non-null  object 
 3   Award Date                               29329 non-null  object 
 4   Fiscal Year                              29329 non-null  int64  
 5   Command                                  29329 non-null  object 
 6   Sub Command                              29329 non-null  object 
 7   Organization                             29329 non-null  object 
 8   Office                                   29329 non-null  object 
 9   Office Id                                29329 non-null  object 
 10  Army Hierarchy                           29024

In [ ]:
# Display the first few rows of the dataframe
df.head()

In [ ]:
# Display basic statistics of the dataframe
df.describe()

In [ ]:
#Check for missing values
missing_values = df.isnull().sum()
print('Missing Values:\n', missing_values)

In [ ]:
# Drop rows with missing values
# df = df.dropna()

In [ ]:
# Display the correlation matrix of the dataframe
# df.corr()

In [ ]:
# Identify incorrect data types
print("Data Types:\n", df.dtypes)

In [ ]:
# Convert dates to a consistent format
df['Award Date'] = pd.to_datetime(df['Award Date'], errors='coerce')
df['Expiration'] = pd.to_datetime(df['Expiration'], errors='coerce')
print("Award Date Data Type:", df['Award Date'].dtype)
print("Expiration Date Data Type:", df['Expiration'].dtype)

In [ ]:
# Convert 'Small Business Dollars' to numeric type if it is not already
df['SB Dollars'] = pd.to_numeric(df['SB Dollars'], errors='coerce')

df.info()

In [ ]:
# Find outliers in numeric columns. An outlier would be a negative value in the 'Small Business Dollars' column
outliers = df[df['SB Dollars'] < 0]
print("Outliers:\n", outliers)

In [ ]:
# Detect duplicate recrods in dataframe
duplicates = df[df.duplicated(subset=['Contract No', "NAICS"], keep=False)]
print("Duplicate Records:\n", duplicates)

In [ ]:
# Standardize text data
df['Contract No'] = df['Contract No'].str.upper()
print("Standardized Contract No:\n", df['Contract No'])

In [ ]:
# Convert 'NAICS' column to string type
df['NAICS'] = df['NAICS'].astype(str)
df.info()

In [ ]:
# Validate data ranges to ensure they are within expected ranges
invalid_naics = df[~df['NAICS'].str.match(r'^\d{6}$')] # Check if NAICS codes are 6 digits
print("Invalid NAICS Codes:\n", invalid_naics)

In [ ]:
# Ensure NAICS values are only six digits and remove any values after six digits
df['NAICS'] = df['NAICS'].str[:6]
print("Invalid NAICS Codes Corrected:\n", df['NAICS'])

In [ ]:
invalid_naics = df[~df['NAICS'].str.match(r'^\d{6}$')] # Check if NAICS codes are 6 digits
print("Invalid NAICS Codes:\n", invalid_naics)

In [ ]:
# Count unique values in a column
unique_naics = df['NAICS'].nunique()

# Count unique NAICS in Fiscal Year 2024 where fiscal year is 10/1/2023 - 9/30/2024
fy_2024 = df[(df['Award Date'] >= '2023-10-01') & (df['Award Date'] <= '2024-09-30')]

# Count unique NAICS in Fiscal Year 2023
fy_2023 = df[(df['Award Date'] >= '2022-10-01') & (df['Award Date'] <= '2023-09-30')]
unique_fy23_naics = fy_2023['NAICS'].nunique()

# Count unique NAICS in Fiscal Year 2022
fy_2022 = df[(df['Award Date'] >= '2021-10-01') & (df['Award Date'] <= '2022-09-30')]
unique_fy22_naics = fy_2022['NAICS'].nunique()

unique_fy24_naics = fy_2024['NAICS'].nunique()
print("Unique NAICS Codes:", unique_naics)
print("Unique NAICS Codes in FY 2024:", unique_fy24_naics)
print("Unique NAICS Codes in FY 2023:", unique_fy23_naics)
print("Unique NAICS Codes in FY 2022:", unique_fy22_naics)

In [ ]:
# Identify how many instances each NAICS code had where it was labeled as SB in the "Size Status" column
sb_naics = df[df['Size Status'] == 'SB']
sb_naics_count = sb_naics['NAICS'].value_counts()
# only dispaly NAICS code with a count greater than 2
sb_naics_count = sb_naics_count[sb_naics_count > 2]
print("SB NAICS Codes:\n", sb_naics_count)
# print("SB NAICS Codes:\n", sb_naics_count)

In [ ]:
#Check for inconsistent formatting across NAICS codes to ensure no more than 6 digits
inconsistent_naics = df[~df['NAICS'].str.match(r'^\d{6}$')]
print("Inconsistent NAICS Codes:\n", inconsistent_naics)

In [ ]:
# convert contract number dtype to str dtype
df['Contract No'] = df['Contract No'].astype(str)
df.info()

In [ ]:
# Convert 'Months Remaining' to integer type
df['Months Remaining'] = pd.to_numeric(df['Months Remaining'], errors='coerce')
df.dtypes

In [ ]:
inconsistent_contract_no = df[~df['Contract No'].str.match(r'^\d{13}$')] # Check if Contract No. codes are 13 digits
print("Inconsistent Contract Numbers:\n", inconsistent_contract_no)

In [ ]:
# Identify rows with multiple issues
issues = df.isnull().any(axis=1) | (df['SB Dollars'] < 0) | (~df['NAICS'].str.match(r'^\d{6}$')) | (~df['Contract No'].str.match(r'^\d{13}$'))
print("Rows with Issues:\n", df[issues])